# Dark Souls II Reviews (2025)

In [1]:
from get_data import *

In [2]:
reviews = get_data()

## Topic Modeling:
- Exploring certain aspects on why people like the game
    - Also get critiques of the game in positive reviews (if any but there sure is considering DS2's reputation in the community)

- Exploring why people don't like the game:
    - Also get positive aspects within this subset of the reviews

Splitting the reviews by how many do and don't recommend buying the game:

In [3]:
pos_reviews = reviews[reviews['voted_up'] == True]
neg_reviews = reviews[reviews['voted_up'] == False]

In [4]:
pos_reviews.shape, neg_reviews.shape

((55777, 6), (11222, 6))

### BERTopic:

In [5]:
from bertopic import BERTopic

In [6]:
# topic_model = BERTopic(verbose=True)

In [7]:
docs = reviews.review.to_list()
pos_docs = pos_reviews.review.to_list()
neg_docs = neg_reviews.review.to_list()

In [8]:
# topic_model.fit(docs)

# topic_model.save("models/bertopic_model")
topic_model = BERTopic.load("models/bertopic_model")

In [9]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,26318,-1_play_game_like_get,"[play, game, like, get, recommend, series, rea...",[know thing hate really want reach conclusion ...
1,0,1999,0_ow_show_till_,"[ow, show, till, , , , , , , ]","[, , show]"
2,1,1360,1_attack_area_enemy_player,"[attack, area, enemy, player, weapon, fight, i...",[begin would like join chorus could estus drin...
3,2,1306,2_good_goodin_uwu_lik,"[good, goodin, uwu, lik, mm, noice, dat, ol, u...","[good, good, good]"
4,3,438,3_de_que_jogo_el,"[de, que, jogo, el, la, um, juego, se, por, novo]",[har un breve resumen para que se haga muy ted...
...,...,...,...,...,...
864,863,10,863_tenderizer_ever_meat_nut,"[tenderizer, ever, meat, nut, anybody, id, sla...","[one bad game ever play, one bad game ive ever..."
865,864,10,864_beginner_maj_beginers_whomever,"[beginner, maj, beginers, whomever, sry, total...",[great however beginner series would much reco...
866,865,10,865_grind_average_paint_spare,"[grind, average, paint, spare, leaderboards, a...",[graphic forget reality beautiful good decent ...
867,866,10,866_back_iiiiiit_iz_disappointing,"[back, iiiiiit, iz, disappointing, hear, wait,...","[back, back, back back back back back back bac..."


In [10]:
topic_model = topic_model.reduce_topics(docs, nr_topics=30)
topic_model.visualize_topics()

In [11]:
topic_model.visualize_hierarchy()

### Positive reviews:

In [12]:
# pos_topic_model = BERTopic(verbose=True)

In [13]:
# pos_topic_model.fit(pos_docs)

# pos_topic_model.save("models/bertopic_pos_model")
pos_topic_model = BERTopic.load("models/bertopic_pos_model")

In [14]:
pos_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,20377,-1_play_game_series_love,"[play, game, series, love, recommend, like, ge...",[unloved little brother maybe family still ii ...
1,0,1669,0_brolaire_umm_ow_praise,"[brolaire, umm, ow, praise, forgot, aaa, hello...","[would praise, ow, brolaire would praise]"
2,1,1270,1_good_mm_dat_ol,"[good, mm, dat, ol, af, lol, dks, thats, pleas...","[good, good, good]"
3,2,1037,2_iron_area_player_attack,"[iron, area, player, attack, enemy, item, weap...",[let preface review say lot fearmongering relu...
4,3,714,3_dlc_version_dx_upgrade,"[dlc, version, dx, upgrade, dlcs, season, plac...",[pro dlcs harder enemy due placement amount ag...
...,...,...,...,...,...
736,735,10,735_port_everthing_extremly_undead,"[port, everthing, extremly, undead, terrimanti...",[pc port great lag everthing high must buy wan...
737,736,10,736_gud_iz_whered_donut,"[gud, iz, whered, donut, relly, ply, yeh, amir...","[gud donut ply, iz gud, lol iz gud]"
738,737,10,737_criticism_criticist_polite_guideline,"[criticism, criticist, polite, guideline, unwa...",[deserve criticism still solid opinion worth a...
739,738,10,738_surving_giveup_nervs_newbs,"[surving, giveup, nervs, newbs, trial, umm, fo...",[one painfully challenge game ever play also o...


In [15]:
pos_topic_model = pos_topic_model.reduce_topics(pos_docs, nr_topics=30)
pos_topic_model.visualize_topics()

In [16]:
pos_topic_model.visualize_hierarchy()

### Negative reviews:

In [17]:
# neg_topic_model = BERTopic(verbose=True)

In [18]:
# neg_topic_model.fit(neg_docs)

# neg_topic_model.save("models/bertopic_neg_model")
neg_topic_model = BERTopic.load("models/bertopic_neg_model")

In [19]:
neg_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4113,-1_fuck_play_like_good,"[fuck, play, like, good, get, bad, game, make,...",[tldr even fan series find control gameplay an...
1,0,1051,0_controller_pc_keyboard_mouse,"[controller, pc, keyboard, mouse, port, contro...","[good pc play like xbox pc control hard, look ..."
2,1,639,1_enemy_area_bos_player,"[enemy, area, bos, player, attack, fight, time...",[couple high point majority low low soulsborne...
3,2,343,2_xddddddddddddddddddddddddddddddddddddddddddd...,[xdddddddddddddddddddddddddddddddddddddddddddd...,[ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh...
4,3,160,3_bad_ever_far_terrible,"[bad, ever, far, terrible, tarkov, make, conce...","[bad bad, bad bad, bad compare hate much bad b..."
...,...,...,...,...,...
143,142,11,142_parry_parrying_queen_rng,"[parry, parrying, queen, rng, bause, bcs, vill...",[honest think skip worth many useless enemy we...
144,143,11,143_masochist_mentally_mannerism_autism,"[masochist, mentally, mannerism, autism, chall...","[best leave masochist, think enjoyable either ..."
145,144,11,144_shame_ashamed_spoiler_yikes,"[shame, ashamed, spoiler, yikes, rip, sad, nam...","[shame call, shame, shame really]"
146,145,10,145_zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz_wwwwwwwww...,"[zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz, wwwwwwwwwww...",[lllllllllllllllllllllllllllllllllllllllllllll...


In [20]:
neg_topic_model = neg_topic_model.reduce_topics(neg_docs, nr_topics=30)
neg_topic_model.visualize_topics()

In [21]:
neg_topic_model.visualize_hierarchy()

### Metrics:

In [22]:
dictionary = Dictionary(reviews.review.apply(word_tokenize))
pos_dictionary = Dictionary(pos_reviews.review.apply(word_tokenize))
neg_dictionary = Dictionary(neg_reviews.review.apply(word_tokenize))

In [23]:
metrics(topic_model, reviews.review, dictionary)

TypeError: metrics() missing 1 required positional argument: 'dictionary'

In [ ]:
metrics(pos_topic_model, pos_reviews.review, pos_dictionary)

In [ ]:
metrics(neg_topic_model, neg_reviews.review, neg_dictionary)